In [1]:
-- Welcome to your new notebook
-- Type here in the cell editor to add code!
CREATE SCHEMA IF NOT EXISTS SalesLakehouse.silver

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
CREATE MATERIALIZED VIEW IF NOT EXISTS SalesLakehouse.silver.sales_consolidated
(
CONSTRAINT sales_quantity_chk CHECK (Quantity!=0) ON MISMATCH DROP
)
AS SELECT 
SV.Agent_ID, 
OV.Order_ID,      
OV.Aging,
OV.Ship_Mode,
OV.Product_Category,
OV.Product,
SV.Sales,
SV.Quantity,
SV.Discount,
SV.Profit,
SV.Shipping_Cost,
SV.Order_Priority,
SV.Customer_ID,
LV.Customer_Name,
LV.Segment,
LV.City,
LV.State,
LV.Country,
LV.Region
FROM SalesLakehouse.bronze.`orders` OV
LEFT JOIN SalesLakehouse.bronze.`sales` SV on OV.ORDER_ID = SV.ORDER_ID
LEFT JOIN SalesLakehouse.bronze.`location` LV on OV.ORDER_ID = LV.ORDER_ID 

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 3, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [3]:
CREATE MATERIALIZED VIEW IF NOT EXISTS SalesLakehouse.silver.sales_data_cleaned
(
CONSTRAINT customer_id_chk CHECK (Customer_ID is not null) ON MISMATCH FAIL
)
AS SELECT 
    Agent_ID,
    Order_ID,
    Aging,
    Ship_Mode,
    Product_Category,
    Product,
    Sales,
    Quantity,
    Discount,
    Profit,
    Shipping_Cost,
    Order_Priority,
    Customer_ID,
    Customer_Name,
    Segment,
    City,
    State,
    Country,
    Region
FROM SalesLakehouse.silver.sales_consolidated

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 4, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [4]:
CREATE SCHEMA IF NOT EXISTS gold

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.OrderPriority
AS SELECT
   distinct(Order_Priority) AS Order_Priority,
   Order_ID
FROM silver.sales_data_cleaned
GROUP BY Order_Priority, Order_ID;

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 6, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [6]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.sales_shipmode
AS SELECT
  Ship_Mode
FROM silver.sales_data_cleaned
GROUP BY Ship_Mode;

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 7, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [7]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.sales_products
AS SELECT
    Product_Category,
    Product
FROM silver.sales_data_cleaned
GROUP BY Product_Category, Product;

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 8, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [8]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.customer_data
AS SELECT
    Customer_ID,
    Order_ID,
    Customer_Name,
    Segment,
    City,
    State,
    Country,
    Region,
    Ship_Mode
FROM silver.sales_data_cleaned

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 9, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [9]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.sales_dates
AS SELECT
  Order_ID,
  Agent_ID
FROM silver.sales_data_cleaned

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 10, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [10]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.total_commissions
AS
SELECT
  Agent_Name,
  Agent_ID,
  Product_Category,
  SUM(Sales) AS Total_Sales,
  SUM(Commission) AS Total_Commission
FROM (
  SELECT 
    s.Order_ID,
    s.Product_Category,
    s.Sales,
    a.Agent_ID,
    a.Agent_Name,
    a.City,
    a.State,
    a.Country,
    ac.Commission_Percentage,
    (s.Sales * ac.Commission_Percentage / 100) AS Commission
  FROM silver.sales_data_cleaned s
  JOIN bronze.agent a
    ON s.Agent_ID = a.Agent_ID
  JOIN bronze.agent_commissions ac
    ON s.Agent_ID = ac.Agent_ID
) AS AgentCommissions
GROUP BY 
Agent_ID, Agent_Name, Product_Category

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 11, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [11]:
CREATE MATERIALIZED VIEW IF NOT EXISTS gold.fact_sales
AS
SELECT 
    d.Order_ID,
    t.Total_Sales AS Price,
    p.Product,
    o.Order_Priority,
    m.Ship_Mode,
    s.Customer_ID
FROM gold.sales_dates d
INNER JOIN gold.customer_data s ON s.Order_ID = d.Order_ID
INNER JOIN gold.total_commissions t ON t.Agent_ID = d.Agent_ID
INNER JOIN gold.sales_products p ON p.Product_Category=t.Product_Category
INNER JOIN gold.sales_shipmode m ON m.Ship_Mode= s.Ship_Mode
INNER JOIN gold.OrderPriority o ON o.Order_ID=d.Order_ID;

StatementMeta(, 39b76f71-bcc0-4406-b7a9-e09f4a8c8fd6, 12, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>